In [6]:
'''Generate Sample'''
import numpy as np
import random

# Step 1: Define parameters for the 1D Ising Model
num_nodes = 10
num_edges = num_nodes - 1

# Randomly assign weights to the edges (between -1 and 1)
edge_weights = np.random.uniform(-1, 1, num_edges)

# Step 2: Solve Ising model
spins = [random.choice([-1,1]) for _ in range(10)] # intialize spin configuration

def spin_config_solver():
    for i in range(num_edges):
        if edge_weights[i] >= 0:
            spins[i+1] = spins[i]
        else:
            spins[i+1] = - spins[i]

spin_config_solver()

def hamiltonian_solver():
    hamiltonian = 0
    for i in range(num_edges):
        hamiltonian += - edge_weights[i] * spins[i] * spins[i+1]
    return hamiltonian # should equal to the opposite value of the absolute sum of "edge_weights" list

hamiltonian = hamiltonian_solver()




[ 0.80454089 -0.25717469 -0.55207389 -0.29244286 -0.24720146 -0.38260707
  0.96343091  0.33503451 -0.74873118]
[-1, -1, 1, -1, 1, -1, 1, 1, 1, -1]
-4.583237447027309
-4.583237447027309


Generate 100 samples of above, store edge_weights, spins, and hamiltonian into a pandas table.

In [ ]:
print(edge_weights)
print(spins)
print(hamiltonian)
print(-sum(abs(x) for x in edge_weights))
